In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

## Dataset

In [85]:
df=pd.read_csv("/content/Bitcoin_data.csv", delimiter=';')

In [3]:
len(df)

5560

In [4]:
df.head(5)

,timeOpen,timeClose,timeHigh,timeLow,name,open,high,low,close,volume,marketCap,timestamp
0,2025-10-02T00:00:00.000Z,2025-10-02T23:59:59.999Z,2025-10-02T19:16:00.000Z,2025-10-02T06:03:00.000Z,2781,118652.385896,121086.407241,118383.158156,120681.259723,7.141516e+10,2.404477e+12,2025-10-02T23:59:59.999Z
1,2025-10-01T00:00:00.000Z,2025-10-01T23:59:59.999Z,2025-10-01T23:59:00.000Z,2025-10-01T00:28:00.000Z,2781,114057.592183,118648.928588,113981.395969,118648.928588,7.132868e+10,2.364529e+12,2025-10-01T23:59:59.999Z
2,2025-09-30T00:00:00.000Z,2025-09-30T23:59:59.999Z,2025-09-30T01:16:00.000Z,2025-09-30T10:04:00.000Z,2781,114396.520241,114836.615425,112740.564747,114056.083647,5.898633e+10,2.272963e+12,2025-09-30T23:59:59.999Z
3,2025-09-29T00:00:00.000Z,2025-09-29T23:59:59.999Z,2025-09-29T20:45:00.000Z,2025-09-29T07:27:00.000Z,2781,112117.878794,114473.569892,111589.950680,114400.386428,6.000015e+10,2.279699e+12,2025-09-29T23:59:59.999Z
4,2025-09-28T00:00:00.000Z,2025-09-28T23:59:59.999Z,2025-09-28T23:10:00.000Z,2025-09-28T12:50:00.000Z,2781,109681.947300,112375.482143,109236.947744,112122.639151,3.337105e+10,2.234221e+12,2025-09-28T23:59:59.999Z


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5560 entries, 0 to 5559
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timeOpen   5560 non-null   object 
 1   timeClose  5560 non-null   object 
 2   timeHigh   5560 non-null   object 
 3   timeLow    5560 non-null   object 
 4   name       5560 non-null   int64  
 5   open       5560 non-null   float64
 6   high       5560 non-null   float64
 7   low        5560 non-null   float64
 8   close      5560 non-null   float64
 9   volume     5560 non-null   float64
 10  marketCap  5560 non-null   float64
 11  timestamp  5560 non-null   object 
dtypes: float64(6), int64(1), object(5)
memory usage: 521.4+ KB


In [6]:
df.isnull().sum()

,0
timeOpen,0
timeClose,0
timeHigh,0
timeLow,0
name,0
open,0
high,0
low,0
close,0
volume,0


### Preprocess data: Turning the dataset into univariate data

In [86]:
df["date"] = pd.to_datetime(df["timeClose"]).dt.date

In [87]:
bitcoin_data=df[["date","close"]]

In [88]:
bitcoin_data.set_index("date", inplace=True)
bitcoin_data.rename(columns={"close":"price"}, inplace=True)

/tmp/ipython-input-3330979986.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bitcoin_data.rename(columns={"close":"price"}, inplace=True)


In [89]:
bitcoin_data.head()

,price
date,
2025-10-02,120681.259723
2025-10-01,118648.928588
2025-09-30,114056.083647
2025-09-29,114400.386428
2025-09-28,112122.639151


In [11]:
bitcoin_data.isnull().sum()

,0
price,0


## Model 1: N-Beats

In [90]:
HORIZON=1
WINDOW_SIZE=7

In [91]:
# Custom block layer

from tensorflow import keras
from tensorflow.keras import layers

class NBeatsBlock(layers.Layer):
    def __init__(self,
                 input_size: int,
                 theta_size: int,
                 horizon: int,
                 n_neurons: int,
                 n_layers: int,
                 **kwargs):
      super().__init__(**kwargs)
      self.input_size = input_size
      self.theta_size = theta_size
      self.horizon = horizon
      self.n_neurons = n_neurons
      self.n_layers = n_layers

      self.hidden=[layers.Dense(n_neurons, activation="relu") for _ in range(n_layers)]
      self.theta_layer=layers.Dense(theta_size, activation="linear")

    def call(self, inputs):
      x=inputs
      for layer in self.hidden:
        x=layer(x)
      theta=self.theta_layer(x)
      backast, forecast=theta[:,:self.input_size], theta[:,-self.horizon:]
      return backast, forecast

In [92]:
nbeats_data=bitcoin_data.copy()
for i in range(WINDOW_SIZE):
  nbeats_data[f"Price+{i+1}"]=nbeats_data["price"].shift(periods=i+1)

In [93]:
nbeats_data.head()

,price,Price+1,Price+2,Price+3,Price+4,Price+5,Price+6,Price+7
date,,,,,,,,
2025-10-02,120681.259723,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-10-01,118648.928588,120681.259723,NaN,NaN,NaN,NaN,NaN,NaN
2025-09-30,114056.083647,118648.928588,120681.259723,NaN,NaN,NaN,NaN,NaN
2025-09-29,114400.386428,114056.083647,118648.928588,120681.259723,NaN,NaN,NaN,NaN
2025-09-28,112122.639151,114400.386428,114056.083647,118648.928588,120681.259723,NaN,NaN,NaN


In [94]:
# Train and test splits

X=nbeats_data.dropna().drop("price", axis=1)
y=nbeats_data.dropna()["price"]

split_size=int(len(X)*0.8)
X_train, y_train=X[:split_size], y[:split_size]
X_test, y_test=X[split_size:], y[split_size:]

In [95]:
len(X_train), len(y_train), len(X_test), len(y_test)

(4442, 4442, 1111, 1111)

In [96]:
# Batch and prefetch data

train_data_feature=tf.data.Dataset.from_tensor_slices(X_train)
train_data_label=tf.data.Dataset.from_tensor_slices(y_train)
train_data=tf.data.Dataset.zip((train_data_feature, train_data_label))

test_data_feature=tf.data.Dataset.from_tensor_slices(X_test)
test_data_label=tf.data.Dataset.from_tensor_slices(y_test)
test_data=tf.data.Dataset.zip((test_data_feature, test_data_label))

train_data=train_data.batch(batch_size=1024).prefetch(tf.data.AUTOTUNE)
test_data=test_data.batch(batch_size=1024).prefetch(tf.data.AUTOTUNE)

train_data, test_data

(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 7), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 7), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>)

In [97]:
len(train_data), len(test_data)

(5, 2)

In [98]:
# Hyperparameters for N-Beats

N_LAYERS=4
N_NUERONS=512
N_EPOCHS=500
N_STACKS=30
INPUT_SIZE=WINDOW_SIZE*HORIZON
THETA_SIZE=WINDOW_SIZE+HORIZON

In [99]:
# Model
n_beats_block=NBeatsBlock(input_size=INPUT_SIZE,
                          theta_size=THETA_SIZE,
                          horizon=HORIZON,
                          n_neurons=N_NUERONS,
                          n_layers=N_LAYERS,
                          name="N_Beats_Model")

stack_input=tf.keras.layers.Input(shape=(INPUT_SIZE,), name="stack_input", dtype=tf.float32)
residuals, forecast=n_beats_block(stack_input)
for i, _ in enumerate(range(N_STACKS-1)):
  backast, block_forecast=n_beats_block(residuals)
  residuals=layers.add([residuals, backast], name=f"residuals_{i}")
  forecast=layers.add([forecast, block_forecast], name=f"forecast_{i}")
nbeats=tf.keras.Model(inputs=stack_input, outputs=forecast, name="N_Beats")

In [100]:
nbeats.summary()

Model: "N_Beats"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ stack_input         │ (None, 7)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ N_Beats_Model       │ [(None, 7),       │    796,168 │ stack_input[0][0… │
│ (NBeatsBlock)       │ (None, 1)]        │            │ N_Beats_Model[0]… │
│                     │                   │            │ residuals_0[0][0… │
│                     │                   │            │ residuals_1[0][0… │
│                     │                   │            │ residuals_2[0][0… │
│                     │                   │            │ residuals_3[0][0… │
│                     │                   │            │ residuals_4[0][0… │
│                     │                   │            │ residuals_5[0][0… │
│                     │                   │            │ residuals_6[0][0… │
│                     │                   │            │ residuals_7[0][0… │
│                     │                   │            │ residuals_8[0][0… │
│                     │                   │            │ residuals_9[0][0… │
│                     │                   │            │ residuals_10[0][… │
│                     │                   │            │ residuals_11[0][… │
│                     │                   │            │ residuals_12[0][… │
│                     │                   │            │ residuals_13[0][… │
│                     │                   │            │ residuals_14[0][… │
│                     │                   │            │ residuals_15[0][… │
│                     │                   │            │ residuals_16[0][… │
│                     │                   │            │ residuals_17[0][… │
│                     │                   │            │ residuals_18[0][… │
│                     │                   │            │ residuals_19[0][… │
│                     │                   │            │ residuals_20[0][… │
│                     │                   │            │ residuals_21[0][… │
│                     │                   │            │ residuals_22[0][… │
│                     │                   │            │ residuals_23[0][… │
│                     │                   │            │ residuals_24[0][… │
│                     │                   │            │ residuals_25[0][… │
│                     │                   │            │ residuals_26[0][… │
│                     │                   │            │ residuals_27[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ residuals_0 (Add)   │ (None, 7)         │          0 │ N_Beats_Model[0]… │
│                     │                   │            │ N_Beats_Model[1]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ residuals_1 (Add)   │ (None, 7)         │          0 │ residuals_0[0][0… │
│                     │                   │            │ N_Beats_Model[2]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ residuals_2 (Add)   │ (None, 7)         │          0 │ residuals_1[0][0… │
│                     │                   │            │ N_Beats_Model[3]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ residuals_3 (Add)   │ (None, 7)         │          0 │ residuals_2[0][0… │
│                     │                   │            │ N_Beats_Model[4]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ residuals_4 (Add)   │ (None, 7)         │          0 │ residuals_3[0][0… │
│                     │                   │            │ N_Beats_Model[5]… │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 796,168 (3.04 MB)

 Trainable params: 796,168 (3.04 MB)

 Non-trainable params: 0 (0.00 B)

In [101]:
# Compile the model
nbeats.compile(loss="mae",
               optimizer=tf.keras.optimizers.Adam(0.001),
               metrics=["mae", "mse"])

In [102]:
# Fit the model
nbeats.fit(train_data,
           epochs=N_EPOCHS,
           validation_data=test_data,
           callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=200, restore_best_weights=True),
                      tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=100, verbose=1)])

Epoch 1/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 21s 3s/step - loss: 186162.0469 - mae: 186162.0469 - mse: 132603084800.0000 - val_loss: 4.5727 - val_mae: 4.5727 - val_mse: 95.5824 - learning_rate: 0.0010
Epoch 2/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 8843.2100 - mae: 8843.2100 - mse: 158536736.0000 - val_loss: 74.4750 - val_mae: 74.4750 - val_mse: 22016.3281 - learning_rate: 0.0010
Epoch 3/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 96175.1484 - mae: 96175.1484 - mse: 26712995840.0000 - val_loss: 7.9962 - val_mae: 7.9962 - val_mse: 340.4132 - learning_rate: 0.0010
Epoch 4/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 17188.4336 - mae: 17188.4336 - mse: 577149440.0000 - val_loss: 7.5247 - val_mae: 7.5247 - val_mse: 298.8871 - learning_rate: 0.0010
Epoch 5/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 12317.0117 - mae: 12317.0117 - mse: 408376544.0000 - val_loss: 4.1273 - val_mae: 4.1273 - val_mse: 49.8990 - learning_rate: 0.0010
Epoch 6/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms

In [103]:
# Evaluate the model
n_beats_pred=nbeats.predict(test_data)

2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step


In [104]:
tf.squeeze(n_beats_pred,axis=1)

<tf.Tensor: shape=(1111,), dtype=float32, numpy=
array([98.269485 , 97.44167  , 95.42622  , ...,  1.5294461,  1.5301267,
        1.5273975], dtype=float32)>

In [105]:
# Evaluate the results
def evaluate_preds(y_true, y_pred):
  #y_true=tf.cast(y_true, dtype=tf.float32)
  #y_pred=tf.cast(y_pred, dtype=tf.float32)

  mae=tf.keras.metrics.MeanAbsoluteError()(y_true, y_pred)
  mse=tf.keras.metrics.MeanSquaredError()(y_true, y_pred)
  rmse=np.sqrt(mse)
  mape=tf.keras.metrics.MeanAbsolutePercentageError()(y_true, y_pred)

  return {"mae": float(mae),
          "mse":float(mse),
          "rmse":float(rmse),
          "mape":float(mape)}

In [106]:
n_beats_evaluation=evaluate_preds(y_test, tf.squeeze(n_beats_pred))
n_beats_evaluation

{'mae': 2.2790067195892334,
 'mse': 44.00614547729492,
 'rmse': 6.6337127685546875,
 'mape': 260.18157958984375}

## Model 2: Ensemble

In [18]:
def ensemble_model(horizon=HORIZON,
                   train_data=train_data,
                   num_iter=5,
                   test_data=test_data,
                   num_epochs=1000,
                   loss_fns=["mae","mse"]):
  ensemble_models=[]
  for i in range(num_iter):
    for loss_fn in loss_fns:
      model = tf.keras.Sequential([
          layers.Dense(128, kernel_initializer="he_normal", activation="relu"),
          layers.Dense(128, kernel_initializer="he_normal", activation="relu"),
          layers.Dense(horizon)
      ])
      model.compile(loss=loss_fn,
                             optimizer=tf.keras.optimizers.Adam(0.001),
                             metrics=["mae", "mse"])
      model.fit(train_data,
                         epochs=num_epochs,
                         validation_data=test_data,
                         callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=200, restore_best_weights=True),
                                    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=100, verbose=1)])
      ensemble_models.append(model)
  return ensemble_models

In [19]:
ensemble_models_bitcoin=ensemble_model(train_data=train_data,
                                       num_iter=5,
                                       test_data=test_data)

Streaming output truncated to the last 5000 lines.
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 701.3232 - mae: 701.3232 - mse: 1570161.6250 - val_loss: 0.9793 - val_mae: 0.9793 - val_mse: 17.4168 - learning_rate: 1.0000e-04
Epoch 329/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 698.8194 - mae: 698.8194 - mse: 1582550.1250 - val_loss: 0.9698 - val_mae: 0.9698 - val_mse: 17.3020 - learning_rate: 1.0000e-04
Epoch 330/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 698.6754 - mae: 698.6754 - mse: 1565411.8750 - val_loss: 0.9811 - val_mae: 0.9811 - val_mse: 17.4577 - learning_rate: 1.0000e-04
Epoch 331/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 699.8808 - mae: 699.8808 - mse: 1587069.8750 - val_loss: 0.9699 - val_mae: 0.9699 - val_mse: 17.3253 - learning_rate: 1.0000e-04
Epoch 332/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 700.8392 - mae: 700.8392 - mse: 1569134.7500 - val_loss: 0.9810 - val_mae: 0.9810 - val_mse: 17.4450 - learning_rate: 1.0000e-04
Epoch 333/1000

In [44]:
# Make predictions with ensemble model

def ensemble_predictions(ensemble_models, test_data):
  ensemble_preds=[]
  for model in ensemble_models:
    preds=model.predict(test_data)
    ensemble_preds.append(preds)
  return tf.constant(tf.squeeze(ensemble_preds))

In [45]:
ensemble_preds = ensemble_predictions(ensemble_models_bitcoin,test_data)
ensemble_preds

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


<tf.Tensor: shape=(10, 1111), dtype=float32, numpy=
array([[ 1.02117500e+02,  9.87457123e+01,  9.36247177e+01, ...,
         3.29898894e-02,  1.84104443e-02,  9.91645455e-03],
       [ 1.02895210e+02,  9.91981125e+01,  9.55162888e+01, ...,
         1.66112319e-01,  1.46813810e-01,  1.42536551e-01],
       [ 1.02377739e+02,  9.93990784e+01,  9.52153931e+01, ...,
         8.85775536e-02,  7.69128650e-02,  7.07429796e-02],
       ...,
       [ 1.01974136e+02,  9.63740387e+01,  8.99469147e+01, ...,
         1.12603024e-01,  1.02792665e-01,  8.85209963e-02],
       [ 1.02282417e+02,  1.00382103e+02,  9.47600250e+01, ...,
        -1.51541993e-01, -1.53989568e-01, -1.67446911e-01],
       [ 1.01968544e+02,  9.94841232e+01,  9.37380371e+01, ...,
         1.65794313e-01,  1.56528533e-01,  1.39217794e-01]], dtype=float32)>

In [46]:
ensemble_preds=evaluate_preds(y_test, ensemble_preds)
ensemble_preds

{'mae': 1.0465128421783447,
 'mse': 19.136085510253906,
 'rmse': 4.374481201171875,
 'mape': 14.398977279663086}

## Model 3: ARIMA

In [126]:
from statsmodels.tsa.arima.model import ARIMA, ARIMAResultsWrapper
import statsmodels

In [55]:
split_size=int(len(X)*0.8)
X_train, y_train=X[:split_size], y[:split_size]
X_test, y_test=X[split_size:], y[split_size:]

In [57]:
arima_model = ARIMA(y_train, order=(5, 1, 0))
arima_model_fit = arima_model.fit()

/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency -1D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency -1D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inf

In [61]:
arima_forecast = len(y_test)
forecast = arima_model_fit.forecast(steps=arima_forecast)

/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [62]:
arima_preds=evaluate_preds(y_test, forecast)
arima_preds

{'mae': 80.6503677368164,
 'mse': 7047.10205078125,
 'rmse': 83.947021484375,
 'mape': 17801.955078125}

In [63]:
arima_model_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                  price   No. Observations:                 4448
Model:                 ARIMA(5, 1, 0)   Log Likelihood              -37067.923
Date:                Sat, 04 Oct 2025   AIC                          74147.847
Time:                        23:22:52   BIC                          74186.247
Sample:                             0   HQIC                         74161.386
                               - 4448                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.0043      0.004     -1.191      0.234      -0.011       0.003
ar.L2         -0.0026      0.007     -0.358      0.721      -0.017       0.011
ar.L3          0.0060      0.008      0.731      0.464      -0.010       0.022
ar.L4          0.0315      0.007      4.426      0.000       0.018       0.045
ar.L5          0.0054      0.007      0.737      0.461      -0.009       0.020
sigma2      1.011e+06   8017.117    126.059      0.000    9.95e+05    1.03e+06
===================================================================================
Ljung-Box (L1) (Q):                  13.43   Jarque-Bera (JB):             29415.24
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.00   Skew:                            -0.30
Prob(H) (two-sided):                  0.00   Kurtosis:                        15.59
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

## Evaluation metrics

In [143]:
evaluation_metrics=pd.DataFrame([n_beats_evaluation, ensemble_preds, arima_preds], index=["N-Beats", "Ensemble", "ARIMA"])
evaluation_metrics.columns=["MAE", "MSE", "RMSE", "MAPE"]
display(evaluation_metrics)

,MAE,MSE,RMSE,MAPE
N-Beats,2.279007,44.006145,6.633713,260.181580
Ensemble,1.046513,19.136086,4.374481,14.398977
ARIMA,80.650368,7047.102051,83.947021,17801.955078
